In [1]:
from osgeo import gdal
from sklearn import svm
import numpy
import time
from osgeo import gdal
from sklearn.externals import joblib
from numpy.core.umath import add, subtract


class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def Band2Array(self):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        array = band.ReadAsArray(0, 0, cols, rows)
        #array [array <-100] =fill
        #array = numpy.where(array ==fill, fill , array * 0.0001)
        array= array.reshape(cols*rows)
        return  array

    def trtData(self,evi,lswi):
        X=[]
        for i in range(len(evi)):
            F=[]
            F.append(evi[i])
            F.append(lswi[i])
            #F.append(dvel[i])
            F.append(subtract(evi[i],lswi[i]))
            X.append(F)
        return X

    def trainResult(self, flood):
        y = []
        for i in range(len(flood)):
            y.append(flood[i])
        return y

    def index2Array(self,fill):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        array = band.ReadAsArray(0, 0, cols, rows)
        array = array.reshape(cols * rows)
        return array

    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Int16)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand

    def cMatrix(self, actual, test):
        F_F=0
        F_N=0
        F_fi=0
        N_F =0
        N_N =0
        N_fi=0
        Fi_F=0
        Fi_N=0
        Fi_fi=0

        for i in range(len(actual)):
            if actual[i]==test[i]:
                if actual[i]==0:
                    N_N += 1
                elif actual[i]==1:
                    F_F += 1
                else:
                    Fi_fi += 1
            else:
                if actual[i]==0:
                    if test[i]==1:
                        N_F += 1
                    else:
                        N_fi+= 1
                elif actual[i]==1:
                    if test[i]==0:
                        F_N +=1
                    else:
                        F_fi +=1
                else:
                    if test[i]==1:
                        Fi_F +=1
                    else:
                        Fi_N +=1

        LIST= [F_F,F_N,F_fi,N_F,N_N,
               N_fi,Fi_F,Fi_N,Fi_fi]

        C_matrix = numpy.array(LIST).reshape(3,3)
        return C_matrix






In [ ]:

fill=-999
path="/home/faizan/Desktop/full/"


#assigning names training data

Tevi=path+"evi_training1.tif"
#Tdvel=path+"dvel_training1.tif"
Tlswi=path+"lswi_training1.tif"
Tflood=path+"flood_training1.tif"

#loading image training
limage = Index(Tevi)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
EVI_t = (limage.index2Array(fill))

limage = Index(Tlswi)
LSWI_t = (limage.index2Array(fill))

#limage = Index(Tdvel)
#DVEL_t = (limage.index2Array(fill))

limage = Index(Tflood)
Flood_t = (limage.index2Array(fill))

In [22]:
len(EVI_t)

95665

In [23]:
Tevi=path+"evi_training2.tif"
#Tdvel=path+"dvel_training1.tif"
Tlswi=path+"lswi_training2.tif"
Tflood=path+"flood_training2.tif"

#loading image training
limage = Index(Tevi)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
EVI_t1 = (limage.index2Array(fill))

limage = Index(Tlswi)
LSWI_t1 = (limage.index2Array(fill))

#limage = Index(Tdvel)
#DVEL_t = (limage.index2Array(fill))

limage = Index(Tflood)
Flood_t1 = (limage.index2Array(fill))

In [24]:
evi= numpy.concatenate([EVI_t,EVI_t1])
lswi= numpy.concatenate([LSWI_t,LSWI_t1])
flood= numpy.concatenate([Flood_t,Flood_t1])

In [25]:
# arranging training data
X = (limage.trtData(evi, lswi))
Y = (limage.trainResult(flood))
# SVM machine learning
#clf = svm.SVC()
#clf = clf.fit(X,Y)

#save model

#joblib.dump(clf, '/home/faizan/Desktop/model/modis_trained.pkl')
del Flood_t1, Flood_t, EVI_t, EVI_t1, LSWI_t, LSWI_t1

In [26]:
from sklearn.cross_validation import cross_val_score
score = cross_val_score(clf, X, Y)
score.mean()

0.98270782790894218

In [27]:
flood_res=clf.predict(X)

In [29]:
cMat= limage.cMatrix(flood, flood_res)
cMat

array([[ 85038,   2164,      0],
       [  2145, 179466,      0],
       [     0,      0,  36180]])

In [2]:
clf = joblib.load('/home/faizan/Desktop/model/modis_trained.pkl')
fill=-999

path="/home/faizan/USA_data/8days/pakistan8days/"
np_dir="/home/faizan/Desktop/numpy_dir/"
with open('/home/faizan/USA_data/8days/pakistan8days/list9.txt',"r") as f:
    #g=len(f.readlines())     #print sum(1 for _ in f)
    for line in f:
        l= line.split(".")
        #assigning names test data
        evi=path+str(l[0])+".evi.tif"
        #dvel=path+str(l[0])+".US.dvel.tif"
        lswi=path+str(l[0])+".lswi.tif"
        Oflood = path+str(l[0])+".svm.flood.tif"
        #save model

        #joblib.dump(clf, '/home/faizan/Desktop/full/trained.pkl')

        #Test data
        limage = Index(evi)

        cols = limage.iminfo['cols']
        # Get pixel row number
        rows = limage.iminfo['rows']
        bandnum= limage.iminfo['bandnum']
        EVI = (limage.index2Array(fill))

        limage = Index(lswi)
        LSWI = (limage.index2Array(fill))

        #limage = Index(dvel)
        #DVEL = (limage.index2Array(fill))

        #Data = (limage.trtData(EVI, LSWI, DVEL))
        Data = numpy.vstack([EVI, LSWI])
        del EVI, LSWI

        def argwrapper(args):
            '''
            ラッパー関数
            '''
            return args[0](*args[1:])

        def myfunc(x):
            '''
            並列に計算したい関数
            '''
            return clf.predict([Data[0][x], Data[1][x], subtract(Data[0][x], Data[1][x])])

        start_time = time.time()
        if __name__ == '__main__':
            from multiprocessing import Pool
            p = Pool(6)
            for a in xrange(0, 10):
                func_args = []
                for i in xrange((len(Data[0])*a)/10, (len(Data[0])*(a+1))/10):
                    func_args.append((myfunc, i))
                results = numpy.array(p.map(argwrapper, func_args), dtype=numpy.int16)
                del func_args
                numpy.save(np_dir + "svm_result%02d" % a, results)
                del results
                print("--- %s seconds ---" % (time.time() - start_time))
            del Data
            # Load all array and merge
            # load first array
            ARRAY = numpy.load(np_dir + "svm_result00.npy")

            for a in range(1, 10):
                dummy = numpy.load(np_dir + "svm_result%02d.npy" % (a))
                ARRAY = numpy.vstack([ARRAY, dummy])
            limage.WriteArrayAsImage(Oflood, ARRAY)
            del ARRAY, dummy
f.close()

--- 63.9257981777 seconds ---
--- 128.726298094 seconds ---
--- 192.879475117 seconds ---
--- 257.160576105 seconds ---
--- 322.658303022 seconds ---
--- 386.797775984 seconds ---
--- 450.817746162 seconds ---
--- 514.342998981 seconds ---
--- 577.657387018 seconds ---
--- 641.919533968 seconds ---
--- 63.301003933 seconds ---
--- 129.268961906 seconds ---
--- 195.222319126 seconds ---
--- 258.987114906 seconds ---
--- 325.490432024 seconds ---
--- 390.446150064 seconds ---
--- 454.227670908 seconds ---
--- 518.53579402 seconds ---
--- 584.205034971 seconds ---
--- 649.111774921 seconds ---


In [59]:
limage.WriteArrayAsImage(Oflood, ARRAY)

In [55]:
limage = Index(evi)

cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']

In [56]:
len(ARRAY)

6792000

In [57]:
rows

2000

In [58]:
cols

3396

In [46]:
x=5000
dvel=([Data[0][x], Data[1][x], subtract(Data[0][x], Data[1][x])])

In [47]:
dvel

[0.42161101, 0.17663209, 0.24497892]

In [9]:

fill=-999
path="/home/faizan/USA_data/8days/pakistan8days/"


#assigning names training data

act=path+"A2010321.flood.tif"
#Tdvel=path+"dvel_training1.tif"
tst=path+"A2010321.machineflood.tif"

#act='/home/faizan/USA_data/h10v0/h11v04.A2008153.US.flood.tif'

#tst='/home/faizan/USA_data/h10v0/h11v04.A2008153.svm.flood.tif'

limage = Index(act)
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
ACT = (limage.index2Array(fill))

limage = Index(tst)
Test = (limage.index2Array(fill))


In [10]:
len(Test)

6792000

In [11]:
cMat= limage.cMatrix(ACT, Test)
cMat

array([[1062948,   27538,       0],
       [   2663, 3327648,       0],
       [      0,       0, 2371203]])